## Capitol Evictions: Evictions and Development in Ward 8

### Table of Contents
1. <a href ='#goal'>Problem Statement</a>
2. <a href='#datasources'>Data Sources</a>
3. <a href='#EDA'>Exploratory Data Analysis</a>
4. <a href='#regression'>Regression</a>
5. <a href='#findings'>Analysis and Findings</a>
6. <a href='#rec'>Recommendations</a>

In [10]:
# Import libraries and set defaults
from shapely.geometry import Point, Polygon
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

In [2]:
# color palette options:
# sns.set_palette(sns.color_palette("Blues_r", 8,))
# sns.set_palette("coolwarm", 3)
# sns.set_palette(sns.diverging_palette(10, 220, sep=80, n=8))
# sns.palplot(current_palette) # <- use to view palette

<a id='goal'></a>
### Problem Statement

The goal of this analysis is to gain a better understanding of the relationship between Planned Unit Developments (PUDs) and eviction rates.

A planned unit development refers to ... 

<a id='datasources'></a>
### Data Sources

The Eviction Lab: https://evictionlab.org/

This research uses data from The Eviction Lab at Princeton University, a project directed by Matthew Desmond and designed by Ashley Gromis, Lavar Edmonds, James Hendrickson, Katie Krywokulski, Lillian Leung, and Adam Porton. The Eviction Lab is funded by the JPB, Gates, and Ford Foundations as well as the Chan Zuckerberg Initiative. More information is found at evictionlab.org.


Open Data DC: https://opendata.dc.gov/
 - Planned Unit Developments: https://opendata.dc.gov/datasets/1b3e77aaa6154d1285af639323b0504f_14/data

In [6]:
# import data
puds = pd.read_csv('data/final_datasets/master_puds_blocks.csv')

<a id='collection'></a>
### Data Cleaning

TODO: The Eviction Lab has data at the census tract level. We limited the dataset to 2016 data for census tracts within Ward 8. We also restricted the Open Data DC dataset on PUDs, which was available on the ward level, to Ward 8. We used GoogleMyMaps to overlay a map of D.C. wards with GEOIDs (census tracts) and PUD locations. Using the map as a reference, we manually coded which PUDs and GEOIDs were in Ward 8. 

<a id='EDA'></a>
### Exploratory Data Analysis

<a id='regression'></a>
### Regression

In [11]:
%run python_files/feature_selection_blocks

<a id='findings'></a>
### Analysis and Findings

<a id='recs'></a>
### Recommendations